# Cross-Classification Method

Predicts number of trips in the future. 

- Households are stratified based on selected attributes
- Trip generation rates for each household stratification

*Assumption:* the trip generation rates are stable over time

**Input:** 
- For model estimation: observations of the base-year trip generation rates and household attributes
- For forecasting: predicted future-year household attributes


In [1]:
#### Importing libriaries
import numpy as np 
import pandas as pd


In [2]:
#### Store the url string that hosts our .csv file (data file should be located in the folder csv)
url = "tripv2pub.csv"

#### Read the .csv file using `pandas` libriary and store it as a `pandas` Data Frame
da = pd.read_csv(url)

#### Finally, let's make sure that all columns are displayed
pd.set_option('display.max_columns', None)

In [3]:
#### This week we will be using trip file, let's check that we uploaded the correct one
da.head()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
0,9000013002,1,1,1,900001300201,1,2,1,1,1,2,75,2,3,1,50,15,1435,1450,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,15,2,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020101,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11,4,4,1,20,4,2,5,5,4,3,5,5,3,6,5,4,3
1,9000013002,1,2,2,900001300201,1,2,1,1,1,2,-9,2,3,15,1,10,1605,1615,1,1,3,2,1,2,2,1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,1,1,3.902424,1.608362e+06,0.0,1.569852e+06,90000130020102,3.902424,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,39,2,2,1,1,2,1,7,3,2,3,11,4,4,1,5,5,3,6,5,4,3,20,4,2,5,5,4,3
2,9000013002,2,1,1,900001300202,1,-1,1,1,1,2,10,2,2,1,80,30,700,730,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-1,17.077067,2.208973e+06,0.0,2.274239e+06,90000130020201,17.077067,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,4,2,1,20,4,2,5,5,4,3,40,5,4,6,60,4,3
3,9000013002,2,2,2,900001300202,1,-1,1,1,1,2,10,2,5,12,80,10,740,750,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-9,4.748291,2.208973e+06,0.0,2.274239e+06,90000130020202,4.748291,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,2,2,1,40,5,4,6,60,4,3,5,3,2,4,5,3,2
4,9000013002,2,3,3,900001300202,1,-1,1,1,1,2,-9,2,2,12,1,30,800,830,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,-9,14.098819,2.208973e+06,0.0,2.274239e+06,90000130020203,14.098819,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,2,4,1,5,3,2,4,5,3,2,20,4,2,5,5,4,3


In [4]:
#### Some descriptive statistics! 
da.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
count,3.107400e+04,31074.000000,31074.000000,31074.000000,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.00000,31074.000000,31074.000000,3.107400e+04,3.107400e+04,3.107400e+04,3.107400e+04,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000,31074.000000
mean,9.000116e+09,1.681438,2.437826,2.435702,7.619426e+11,1.048529,1.141308,0.775407,1.722276,3.810066,1.988704,95.176611,1.989026,3.156723,7.185943,26.856311,24.547821,1329.818337,1363.067613,1.041063,1.076945,3.906159,1.997265,1.243644,1.129336,0.185267,-0.226427,-0.644236,-0.880543,-0.954464,-0.976830,-0.993338,-0.999710,0.414140,1.583124,3.333526,0.986774,1.423505,3.786606,-0.802894,-0.986677,82.358853,7.43390,-0.210272,13.965150,8.166137e+06,5.815804e+06,2.302368e+06,9.000116e+13,7.246071,397.951262,5.351805,2.058506,1.654921,1.775729,5.143046,2.652346,1.924117,1.993242,53.722565,7.260636,2.677029,2.821941,2.134453,5.812737,2.292109,3.530926,1.653086,4.137478,1.188968,202211.691543,1.303694,1029.494240,46.800830,1.430617,1.126504,0.855892,2.859014,1.916071,1.268198,4.716097,4.710755,1.503926,1.724657,7.415814,3.039454,3.037910,1.678735,31.759896,4.147133,3.174776,4.903842,31.755970,4.202259,3.226395,31.789985,4.148227,3.176450,4.906900,31.790886,4.200071,3.222533
std,5.829379e+04,0.999429,1.791504,1.789758,3.243517e+11,0.214885,1.341822,0.787571,0.447884,1.966057,0.105681,164.311604,0.104181,1.445374,6.851472,27.561324,46.483373,422.927312,425.447364,0.514696,1.094555,5.054558,3.477865,0.429287,0.982799,1.336076,1.215187,0.911104,0.563497,0.351382,0.254039,0.137048,0.029476,3.387813,0.891605,14.903745,0.833239,0.975625,15.956259,8.739298,0.207546,909.149169,6.99431,4.073497,85.416700,1.083988e+07,9.561997e+06,5.831630e+06,5.829379e+08,27.585135,68.455342,4.189875,0.761605,0.813206,0.417108,2.485935,1.011683,0.264816,0.794912,24.895858,2.861425,11.855930,1.446787,1.078400,3.251390,0.856474,1.552468,1.166259,1.665612,0.391490,20.136521,0.947416,4.982031,20.765643,0.931312,0.872042,0.646882,12.512009,0.277285,0.443029,2.409187,19.318665,0.499993,1.567172,2.445626,1.322290,1.3380

# Selecting trips that we will be predicting
For the purpose of this assignment, let's look into Home-based work trips made on weedays only.

In [5]:
#### Next code choose entries that satisfies our selection criteria
#### First, seelct trips made only on weekdays
#### Second, identify trips that characterised as HBW
dt = da[(da.TDWKND == 2) & (da.TRIPPURP == 1)]



In [6]:
#### Let's see how our data set is different now
dt.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
count,4.333000e+03,4333.000000,4333.000000,4333.000000,4.333000e+03,4333.000000,4333.000000,4333.000000,4333.0,4333.000000,4333.000000,4333.000000,4333.000000,4333.0,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.0,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4.333000e+03,4.333000e+03,4333.0,4.333000e+03,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000,4333.000000
mean,9.000113e+09,1.548581,1.811909,1.812601,8.250276e+11,1.020309,1.290099,0.892684,2.0,3.794369,1.999769,230.085160,1.981999,1.0,1.999308,5.893607,27.084468,1192.292176,1231.836141,1.012001,1.388876,2.931918,1.125087,1.425340,1.172859,0.298177,-0.197554,-0.650127,-0.893607,-0.959843,-0.981999,-0.993076,-1.0,0.052850,1.072236,1.857835,0.924302,1.805677,1.887145,0.113203,-0.967921,45.172398,2.206093,-0.308331,13.452803,8.954795e+06,8.968092e+06,0.0,9.000113e+13,12.740541,396.403670,1.125317,2.159705,1.664205,1.792753,4.986384,2.588045,1.927994,2.103623,52.216247,7.290792,2.853681,2.764136,2.285022,4.634664,2.352412,3.471498,1.747288,4.190861,1.217401,202211.987999,1.764136,1029.210709,45.636511,1.388876,1.029541,1.024233,3.288253,1.921071,1.220632,5.276944,1.241634,1.450727,1.932841,7.496654,2.957304,2.974613,1.231479,32.639049,4.004385,3.070159,4.727441,32.586430,4.082622,3.093469,33.416801,3.991922,3.053773,4.682206,33.437341,4.044542,3.047773
std,5.839736e+04,0.714741,1.217723,1.221710,2.487526e+11,0.141072,1.256961,0.505989,0.0,1.433328,0.015192,255.522426,0.132971,0.0,1.140317,4.483270,25.791707,495.585195,496.132046,0.393089,1.054794,3.500129,1.597132,0.494452,1.011879,1.433406,1.309185,0.956775,0.550739,0.344798,0.231716,0.143971,0.0,1.568746,0.312257,6.862763,0.492598,0.693323,7.007479,23.202528,0.376644,677.862368,1.155995,3.613484,19.340817,1.102478e+07,1.079365e+07,0.0,5.839736e+08,19.468088,67.306593,0.486890,0.849132,0.847355,0.405381,2.456275,1.003648,0.258527,0.869414,24.585044,2.827758,12.512793,1.382931,1.155600,3.179338,0.863605,1.578928,1.222130,1.700928,0.412525,21.217846,0.801797,4.921138,14.490457,0.870113,0.181190,0.166751,13.971288,0.269659,0.414721,1.615631,14.498957,0.497624,1.460582,2.256682,1.340393,1.344014,0.900757,21.809781,2.143238,2.084695,2.574607,23.515387,2.156098,2.225158,22.259794,2.155364,2.114043,2.667275,23.914515,2.196825,2.246219
min,9.000013e+09,1.000

# Let's choose categories for trip stratification

For the purpose of this assignment, let's consider:
- Income
- Urban/rural household location
- Number of adults in the household

We will start with exploring what different entires those columns have

In [7]:
#### Income categories, check codebook!
dt['HHFAMINC'].unique()
#### What should we do with negative entries? 

array([ 6,  8,  7, 11,  9, 10,  5,  3,  4,  2,  1, -7, -8])

In [8]:
#### Unique values at URBRUB column
array_urbur = dt['URBRUR'].unique()
print(array_urbur)

[2 1]


In [9]:
#### umber of adults in participated households
array_numadlt = dt['NUMADLT'].unique()
print(array_numadlt)

[2 3 4 1 5 6 7]


In [10]:
#### Let's remove negative enties for income from the analysis
df = da[(da.TDWKND == 2) & (da.TRIPPURP == 1) & (da.HHFAMINC > 0)] 
array_hhfaninc = df['HHFAMINC'].unique()
print(array_hhfaninc)

[ 6  8  7 11  9 10  5  3  4  2  1]


In [11]:
df.describe()

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
count,4.271000e+03,4271.000000,4271.000000,4271.000000,4.271000e+03,4271.000000,4271.000000,4271.000000,4271.0,4271.000000,4271.000000,4271.000000,4271.000000,4271.0,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.00000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.0,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4.271000e+03,4.271000e+03,4271.0,4.271000e+03,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000,4271.000000
mean,9.000112e+09,1.547647,1.811754,1.812456,8.247820e+11,1.020604,1.295949,0.892063,2.0,3.804495,1.999766,230.372981,1.981737,1.0,1.998595,5.897214,27.102786,1191.665652,1231.221494,1.011239,1.387497,2.931632,1.125029,1.423554,1.168345,0.29431,-0.200890,-0.648560,-0.892063,-0.959260,-0.981737,-0.992976,-1.0,0.053617,1.071412,1.862796,0.923671,1.804730,1.892531,0.103140,-0.969328,45.841723,2.206977,-0.291969,13.431511,8.994604e+06,9.004543e+06,0.0,9.000112e+13,12.710524,396.771506,1.125732,2.151721,1.669398,1.793491,4.986888,2.586982,1.926949,2.094123,52.221962,7.498712,2.876844,2.760712,2.280965,4.608991,2.353079,3.475533,1.741278,4.181456,1.215640,202212.007727,1.760712,1029.209787,45.518380,1.388199,1.029501,1.024584,3.317724,1.919925,1.218684,5.273472,1.271833,1.450948,1.929291,7.498712,2.967221,2.982440,1.232498,32.749707,4.019199,3.084524,4.745025,32.675486,4.102552,3.110279,33.522360,4.004683,3.066729,4.696324,33.521892,4.063217,3.063217
std,5.835915e+04,0.716166,1.221456,1.225488,2.491229e+11,0.142071,1.253559,0.507840,0.0,1.434744,0.015302,255.707730,0.133916,0.0,1.140277,4.482960,25.878322,495.718434,496.277335,0.394773,1.055563,3.502627,1.608178,0.494179,1.016717,1.43338,1.307485,0.958567,0.554572,0.347258,0.233382,0.145010,0.0,1.580081,0.311835,6.911749,0.493783,0.695106,7.057503,23.337529,0.369499,682.742946,1.155886,3.601747,19.371186,1.107044e+07,1.082914e+07,0.0,5.835915e+08,19.498053,67.417251,0.489104,0.845111,0.849740,0.404847,2.460616,1.004840,0.260250,0.864356,24.625730,2.256074,12.601561,1.387249,1.155867,3.171485,0.862918,1.577149,1.219006,1.703170,0.411314,21.272795,0.802433,4.931190,14.507773,0.874350,0.181254,0.167932,14.069964,0.271441,0.413402,1.610459,14.601269,0.497646,1.460859,2.256074,1.338793,1.343183,0.903043,21.861183,2.145330,2.089927,2.577117,23.583844,2.156255,2.230986,22.318504,2.159121,2.120712,2.672662,23.989536,2.198410,2.253435
min,9.000013e+09,1.000000

In [12]:
#### Code in this cell creates 2 grouping and store them in 2 new variables. 
#### This allows us to see groups we are creating! 
#### We will be using them to calculate average value for each group.

ccm_current = df.groupby(['HHFAMINC','URBRUR', 'NUMADLT']).size()
print(ccm_current) # What this grouping does? We can see total number of trips in each group
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")


#### Why we are adding column HOUSEID here? What we will get?
ccm_hh_current = df.groupby(['HHFAMINC','URBRUR', 'NUMADLT','HOUSEID']).size()
print(ccm_hh_current) # What this grouping does? We can see number of hh in each group!
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")


#### Finally, we can use method mean to calculate mean values per each combination of ['HHFAMINC','URBRUR', 'NUMADLT']
ccm_mean_current = df.groupby(['HHFAMINC','URBRUR', 'NUMADLT','HOUSEID']).size().groupby(level = [0, 1, 2]).mean()
print(ccm_mean_current) 

HHFAMINC  URBRUR  NUMADLT
1         1       1           6
                  2           9
                  3           4
          2       1           2
                  2           6
                             ..
11        1       6           6
          2       1           2
                  2          76
                  3          13
                  5           6
Length: 95, dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
HHFAMINC  URBRUR  NUMADLT  HOUSEID   
1         1       1        9000098169    1
                           9000165681    2
                           9000172032    2
                           9000216337    1
                  2        9000048035    1
                                        ..
11        2       3        9000057898    1
                           9000089256    2
                           9000097639    6
                           9000125911    2
                  5        9000164801    6
Length: 1921, dtype: int64
~~~~~~~~~~~~~~~~~

In [13]:
#### Let's look into one enty as example, note the order ['HHFAMINC','URBRUR', 'NUMADLT']
print(ccm_hh_current[1])


URBRUR  NUMADLT  HOUSEID   
1       1        9000098169    1
                 9000165681    2
                 9000172032    2
                 9000216337    1
        2        9000048035    1
                 9000095928    2
                 9000095963    1
                 9000149281    1
                 9000175721    2
                 9000209373    2
        3        9000053946    2
                 9000087816    2
2       1        9000129545    2
        2        9000015238    2
                 9000017013    2
                 9000074475    2
        3        9000089187    3
dtype: int64


In [14]:
print(ccm_hh_current[1, 1])


NUMADLT  HOUSEID   
1        9000098169    1
         9000165681    2
         9000172032    2
         9000216337    1
2        9000048035    1
         9000095928    2
         9000095963    1
         9000149281    1
         9000175721    2
         9000209373    2
3        9000053946    2
         9000087816    2
dtype: int64


In [15]:
print(ccm_hh_current[1, 1, 1])


HOUSEID
9000098169    1
9000165681    2
9000172032    2
9000216337    1
dtype: int64


In [16]:
print(ccm_hh_current[1, 1, 1].size)
print(sum(ccm_hh_current[1, 1, 1]))
print(ccm_mean_current[1, 1, 1])

4
6
1.5


In [17]:
ccm_mean_current.size

95

In [18]:
sum(ccm_mean_current)

231.4311771342524

In [19]:
sum(ccm_mean_current)/ccm_mean_current.size

2.436117654044762

In [20]:
ccm_mean_current.mean()

np.float64(2.4361176540447618)

In [21]:
### We will need these two maps
income_map = {'1' : "Less than $10,000",
              '2' : "$10,000 to $14,999",
              '3' : "$15,000 to $24,999",
              '4' : "$25,000 to $34,999",
              '5' : "$35,000 to $49,999",
              '6' : "$50,000 to $74,999",
              '7' : "$75,000 to $99,999",
              '8' : "$100,000 to $124,999",
              '9' : "$125,000 to $149,999",
              '10' : "$150,000 to $199,999",
              '11' : "$200,000 or more"
             }

urban_map = {'1' : "Urban",
             '2' : "Rural"
            }

In [22]:
#### Let's think how we can calculate average value for each group we have created?
total_trips = 0
miss_hh = 0
included_hh = 0
size = 0
for key_inc in array_hhfaninc: 
    for key_urb in array_urbur:
        for key_adult in array_numadlt:
            try: # Try/except construct allows us to continue program execution even if we have an error! 
                print("Average number of trips made by HHs with income ", income_map[str(key_inc)])
                print("                                   urban/rural ", urban_map[str(key_urb)])
                print("                              number of adults ", key_adult)

                
                average = ccm_mean_current[key_inc, key_urb, key_adult]
                total_trips += ccm_hh_current[key_inc, key_urb, key_adult].size * average
                included_hh += 1
                
                print(f"                                            is {average:.2f}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

                size += ccm_hh_current[key_inc, key_urb, key_adult].size
            
            except:
                print("Something else went wrong XXXXXXXXXXXXXXXXXXXXXX") 
                miss_hh += 1 # This count allows us to know, how many groups in the future will not have inf in current data




Average number of trips made by HHs with income  $50,000 to $74,999
                                   urban/rural  Rural
                              number of adults  2
                                            is 2.04
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Average number of trips made by HHs with income  $50,000 to $74,999
                                   urban/rural  Rural
                              number of adults  3
                                            is 2.58
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Average number of trips made by HHs with income  $50,000 to $74,999
                                   urban/rural  Rural
                              number of adults  4
                                            is 2.00
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Average number of trips made by HHs with income  $50,000 to $74,999
                                   urban/rural  Rural
                              number of adults  1
         

In [23]:
print ("Number of HH groups with valid data: ", included_hh)
print ("Number of HH groups without data: ", miss_hh)

print (f"Number of trips based on available groups:  {total_trips:.2f}")
print (f"Average number of trips per group:  {ccm_mean_current.mean():.2f}")
print (f"Total number of trips assuming average number of trips for missed groups: {(total_trips + miss_hh * ccm_mean_current.mean()):.2f}")                

print ("Number of hoseholds accounted for: ", size)

Number of HH groups with valid data:  95
Number of HH groups without data:  59
Number of trips based on available groups:  4271.00
Average number of trips per group:  2.44
Total number of trips assuming average number of trips for missed groups: 4414.73
Number of hoseholds accounted for:  1921


# Creating future demand

Where we get data? 

For the future demand assume that 
- 10% of HH move from rural to urban areas, 5% will move from urban to rural
- Among current trips, income of 30% of HH will increase one level and 30% decrease one level (randomly)
- 20% of HH will have 1 more adult and 20% will have 1 less adult

In [24]:
#### Build future demand columns, start from creating new columns based on existing
da["NEWHHFAMINC"] = da.loc[:, "HHFAMINC"]
da["NEWNUMADLT"] = da.loc[:, "NUMADLT"]
da["NEWURBRUR"] = da.loc[:, "URBRUR"]

In [25]:
df_1 = da[(da.TDWKND == 2) & (da.TRIPPURP == 1) & (da.HHFAMINC > 0)] 

In [26]:
#### Let's check that we have new columns
#### Note that ID of each column are not sequential numbers! 
df_1.head(10)

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,NEWHHFAMINC,NEWNUMADLT,NEWURBRUR
43,9000013081,2,3,3,900001308102,1,2,1,2,5,2,107,2,1,1,10,13,1430,1443,1,2,4,1,2,1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,3,2,4.364201,4.797160e+06,8.235239e+06,0.0,90000130810203,4.364201,344.9,1,2,2,2,1,1,2,2,13,6,1,2,3,10,3,3,4,6,2,202202,1,1022,60,1,1,1,1,2,1,3,-1,1,4,6,1,1,1,20,2,1,2,30,2,1,20,2,1,2,30,3,2,6,2,2
44,9000013081,2,4,4,900001308102,1,2,1,2,5,2,-9,2,1,3,1,13,1630,1643,1,2,4,1,2,1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,1,4.364201,4.797160e+06,8.235239e+06,0.0,90000130810204,4.364201,344.9,1,2,2,2,1,1,2,2,13,6,1,2,3,10,3,3,4,6,2,202202,1,1022,60,1,1,1,1,2,1,3,-1,1,4,6,1,1,1,20,2,1,2,30,3,2,20,2,1,2,30,2,1,6,2,2
75,9000013167,3,1,1,900001316702,1,-1,1,2,2,2,530,2,1,1,10,15,715,730,1,2,3,1,2,2,1,2,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,3,-1,5.173400,4.872713e+06,4.745249e+06,0.0,90000131670301,5.173400,350.2,1,3,1,1,1,1,2,3,11,8,1,4,2,4,1,5,1,4,1,202202,2,1021,33,2,1,1,1,2,2,5,-1,2,3,8,4,4,1,30,5,4,6,30,5,4,20,4,3,6,5,4,2,8,3,1
76,9000013167,3,2,2,900001316702,1,-1,1,2,2,2,-9,2,1,3,1,15,1620,1635,1,2,3,1,2,2,1,2,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,1,-9,4.746426,4.872713e+06,4.745249e+06,0.0,90000131670302,4.746426,350.2,1,3,1,1,1,1,2,3,11,8,1,4,2,4,1,5,1,4,1,202202,2,1021,33,2,1,1,1,2,2,5,-1,2,3,8,4,4,1,20,4,3,6,5,4,2,30,5,4,6,30,5,4,8,3,1
77,9000013183,1,1,1,900001318301,1,2,1,2,5,2,579,2,1,1,10,6,715,721,1,1,3,1,1,2,2,2,2,2,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,3,1,2.298943,1.334822e+07,1.185463e+07,0.0,90000131830101,2.298943,367.2,1,2,1,2,1,1,2,2,13,7,1,6,2,6,3,3,1,2,1,202203,2,1021,32,2,1,1,1,2,1,5,-1,2,3,7,2,4,1,20,3,2,4,30,1,2,20,4,3,5,30,4,3,7,2,1
78,9000013183,1,2,2,900001318301,1,2,1,2,5,2,-9,2,1,3,1,8,1700,1708,1,1,3,1,1,2,2,2,2,2,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,1,1,2.298943,1.334822e+07,1.185463e+07,0.0,90000131830102,2.298943,367.2,1,2,1,2,1,1,2,2,13,7,1,6,2,6,3,3,1,2,1,202203,2,1021,32,2,1,1,1,2,1,5,-1,2,3,7,4,2,1,20,4,3,5,30,4,3,20,3,2,4,30,1,2,7,2,1
129,9000013284,1,1,1,900001328402,1,2,1,2,6,2,620,2,1,1,10,10,730,740,1,2,4,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,3,2,3.116221,2.423414e+06,4.784481e+06,0.0,90000132840101,3.116221,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,60,1,1,1,1,2,1,6,-1,1,4,11,2,2,1,20,3,2,4,20,1,3,5,3,4,4,5,3,4,11,3,1
131,9000013284,2,1,1,900001328401,1,2,1,2,6,2,470,2,1,1,10,10,700,710,1,1,2,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,3,2,3.081417,2.428866e+06,4.946774e+06,0.0,90000132840201,3.081417,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,58,2,1,1,1,2,1,6,-1,2,2,11,2,2,1,20,3,2,4,20,1,3,20,5,4,6,30,6,4,11,3,1
132,9000013284,2,2,2,900001328401,1,2,1,2,6,2,-9,2,1,3,1,15,1500,1515,1,1,2,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,2,2.986948,2.428866e+06,4.946774e+06,0.0,90000132840202,2.986948,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,58,2,1,1,1,2,1,6,-1,2,2,11,2,2,1,20,5,4,6,30,6,4,20,3,2,4,20,1,3,11,3,1
133,9000013284,3,1,1,900001328403,1,2,1,2,6,2,485,2,1,1,10,10,745,755,1,3,3,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,3,2,3.116221,3.762504e+06,7.889847e+06,0.0,90000132840301,3.116221

In [27]:
#### To be able to modify each participated HH in the df_1 data set, we need to create the array with all House IDs
# Convert the DataFrame's column to a Python list
hh_IDs_array = df_1['HOUSEID'].unique()

print(hh_IDs_array[0:10])

[9000013081 9000013167 9000013183 9000013284 9000013287 9000013360
 9000013380 9000013398 9000013411 9000013541]


In [28]:
#### In this cell we create a new demand by modifying the data in columns we have created

### Get unique HOUSEIDs where URBRUR is 1 (urban)
urban_households = df_1[df_1['URBRUR'] == 1]['HOUSEID'].unique()

### Get unique HOUSEIDs where URBRUR is 2 (rural)
rural_households = df_1[df_1['URBRUR'] == 2]['HOUSEID'].unique()

### Sample 10% of urban households to change from 1 to 2
urban_to_change = pd.Series(urban_households).sample(frac = 0.10, random_state = 42).tolist()

### Sample 5% of rural households to change from 2 to 1
rural_to_change = pd.Series(rural_households).sample(frac = 0.05, random_state = 42).tolist()

### For all rows where HOUSEID is in our urban_to_change list, set NEWURBRUR to 2 
df_1.loc[df_1['HOUSEID'].isin(urban_to_change), 'NEWURBRUR'] = 2

### For all rows where HOUSEID is in our rural_to_change list, set NEWURBRUR to 1
df_1.loc[df_1['HOUSEID'].isin(rural_to_change), 'NEWURBRUR'] = 1




### Prepaer nesseasary arrays for income modification
### Sample 30% of households to increase income
income_to_increase = pd.Series(hh_IDs_array).sample(frac = 0.30, random_state = 42).tolist()

### To create list of HH IDs to decrease the income, let's first create a list that has all HH IDs excluding those that are in array income_to_increase
### Convert the second list to a set for a fast lookup
income_to_increase_set = set(income_to_increase)

### Create the new list by keeping only the items not in the set
hh_IDs_array_1 = [item for item in hh_IDs_array if item not in income_to_increase_set]

### Sample 30% of households to decrease income
income_to_decrease = pd.Series(hh_IDs_array_1).sample(frac = 0.21, random_state = 42).tolist()

### Increase income for households in income_to_increase (with an upper limit of 11) 
# Create a boolean mask for the rows to increase
increase_mask = df_1['HOUSEID'].isin(income_to_increase)

# Apply the increase, using np.minimum to cap the value at 11
df_1.loc[increase_mask, 'NEWHHFAMINC'] = np.minimum(df_1.loc[increase_mask, 'NEWHHFAMINC'] + 1, 11)


### Decrease income for households in income_to_decrease (with a lower limit of 1)
# Create a boolean mask for the rows to decrease
decrease_mask = df_1['HOUSEID'].isin(income_to_decrease)

# Apply the decrease, using np.maximum to set a floor at 1
df_1.loc[decrease_mask, 'NEWHHFAMINC'] = np.maximum(df_1.loc[decrease_mask, 'NEWHHFAMINC'] - 1, 1)




### Prepaer nesseasary arrays for number of adults modification
### Sample 20% of households to increase number of adults
adult_to_increase = pd.Series(hh_IDs_array).sample(frac = 0.20, random_state = 42).tolist()

### To create list of HH IDs to decrease the income, let's first create a list that has all HH IDs excluding those that are in array income_to_increase
### Convert the second list to a set for a fast lookup
adult_to_increase_set = set(adult_to_increase)

### Create the new list by keeping only the items not in the set
hh_IDs_array_1 = [item for item in hh_IDs_array if item not in adult_to_increase_set]

### Sample 20% of households to decrease number of adults
adult_to_decrease = pd.Series(hh_IDs_array_1).sample(frac = 0.16, random_state = 42).tolist()


### Increase adults for selected households
increase_mask = df_1['HOUSEID'].isin(adult_to_increase)
df_1.loc[increase_mask, 'NEWNUMADLT'] = np.minimum(df_1.loc[increase_mask, 'NEWNUMADLT'] + 1, 7)

### Decrease adults for selected households (with a lower limit of 1)
decrease_mask = df_1['HOUSEID'].isin(adult_to_decrease)
df_1.loc[decrease_mask, 'NEWNUMADLT'] = np.maximum(df_1.loc[decrease_mask, 'NEWNUMADLT'] - 1, 1)





In [29]:
#### Let's check the new columns!
df_1.head(10)

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,NEWHHFAMINC,NEWNUMADLT,NEWURBRUR
43,9000013081,2,3,3,900001308102,1,2,1,2,5,2,107,2,1,1,10,13,1430,1443,1,2,4,1,2,1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,3,2,4.364201,4.797160e+06,8.235239e+06,0.0,90000130810203,4.364201,344.9,1,2,2,2,1,1,2,2,13,6,1,2,3,10,3,3,4,6,2,202202,1,1022,60,1,1,1,1,2,1,3,-1,1,4,6,1,1,1,20,2,1,2,30,2,1,20,2,1,2,30,3,2,6,2,2
44,9000013081,2,4,4,900001308102,1,2,1,2,5,2,-9,2,1,3,1,13,1630,1643,1,2,4,1,2,1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,1,4.364201,4.797160e+06,8.235239e+06,0.0,90000130810204,4.364201,344.9,1,2,2,2,1,1,2,2,13,6,1,2,3,10,3,3,4,6,2,202202,1,1022,60,1,1,1,1,2,1,3,-1,1,4,6,1,1,1,20,2,1,2,30,3,2,20,2,1,2,30,2,1,6,2,2
75,9000013167,3,1,1,900001316702,1,-1,1,2,2,2,530,2,1,1,10,15,715,730,1,2,3,1,2,2,1,2,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,3,-1,5.173400,4.872713e+06,4.745249e+06,0.0,90000131670301,5.173400,350.2,1,3,1,1,1,1,2,3,11,8,1,4,2,4,1,5,1,4,1,202202,2,1021,33,2,1,1,1,2,2,5,-1,2,3,8,4,4,1,30,5,4,6,30,5,4,20,4,3,6,5,4,2,8,3,1
76,9000013167,3,2,2,900001316702,1,-1,1,2,2,2,-9,2,1,3,1,15,1620,1635,1,2,3,1,2,2,1,2,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,1,-9,4.746426,4.872713e+06,4.745249e+06,0.0,90000131670302,4.746426,350.2,1,3,1,1,1,1,2,3,11,8,1,4,2,4,1,5,1,4,1,202202,2,1021,33,2,1,1,1,2,2,5,-1,2,3,8,4,4,1,20,4,3,6,5,4,2,30,5,4,6,30,5,4,8,3,1
77,9000013183,1,1,1,900001318301,1,2,1,2,5,2,579,2,1,1,10,6,715,721,1,1,3,1,1,2,2,2,2,2,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,3,1,2.298943,1.334822e+07,1.185463e+07,0.0,90000131830101,2.298943,367.2,1,2,1,2,1,1,2,2,13,7,1,6,2,6,3,3,1,2,1,202203,2,1021,32,2,1,1,1,2,1,5,-1,2,3,7,2,4,1,20,3,2,4,30,1,2,20,4,3,5,30,4,3,7,2,1
78,9000013183,1,2,2,900001318301,1,2,1,2,5,2,-9,2,1,3,1,8,1700,1708,1,1,3,1,1,2,2,2,2,2,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,1,1,2.298943,1.334822e+07,1.185463e+07,0.0,90000131830102,2.298943,367.2,1,2,1,2,1,1,2,2,13,7,1,6,2,6,3,3,1,2,1,202203,2,1021,32,2,1,1,1,2,1,5,-1,2,3,7,4,2,1,20,4,3,5,30,4,3,20,3,2,4,30,1,2,7,2,1
129,9000013284,1,1,1,900001328402,1,2,1,2,6,2,620,2,1,1,10,10,730,740,1,2,4,1,1,2,2,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,3,2,3.116221,2.423414e+06,4.784481e+06,0.0,90000132840101,3.116221,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,60,1,1,1,1,2,1,6,-1,1,4,11,2,2,1,20,3,2,4,20,1,3,5,3,4,4,5,3,4,11,3,1
131,9000013284,2,1,1,900001328401,1,2,1,2,6,2,470,2,1,1,10,10,700,710,1,1,2,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,3,2,3.081417,2.428866e+06,4.946774e+06,0.0,90000132840201,3.081417,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,58,2,1,1,1,2,1,6,-1,2,2,11,2,2,1,20,3,2,4,20,1,3,20,5,4,6,30,6,4,11,3,1
132,9000013284,2,2,2,900001328401,1,2,1,2,6,2,-9,2,1,3,1,15,1500,1515,1,1,2,1,2,1,2,-1,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,2,2.986948,2.428866e+06,4.946774e+06,0.0,90000132840202,2.986948,355.1,1,3,1,2,1,1,2,3,14,11,1,3,3,2,4,6,2,6,1,202202,3,1022,58,2,1,1,1,2,1,6,-1,2,2,11,2,2,1,20,5,4,6,30,6,4,20,3,2,4,20,1,3,11,3,1
133,9000013284,3,1,1,900001328403,1,2,1,2,6,2,485,2,1,1,10,10,745,755,1,3,3,1,2,2,1,-1,-1,-1,-1,-1,-1,-1,0,1,3,1,2,3,-1.0,-1,-1,3,2,3.116221,3.762504e+06,7.889847e+06,0.0,90000132840301,3.116221

In [30]:
#### Here we repeat the same grouping but with new columns
ccm_new = df_1.groupby(['NEWHHFAMINC','NEWURBRUR', 'NEWNUMADLT']).size() # We will not be using this in calcularions
print(ccm_new)
ccm_hh_new = df_1.groupby(['NEWHHFAMINC','NEWURBRUR', 'NEWNUMADLT','HOUSEID']).size()
print(ccm_hh_new)
ccm_mean_new = df_1.groupby(['NEWHHFAMINC','NEWURBRUR', 'NEWNUMADLT','HOUSEID']).size().groupby(level = [0, 1, 2]).mean()
print(ccm_mean_new) 



NEWHHFAMINC  NEWURBRUR  NEWNUMADLT
1            1          1              6
                        2              9
                        3              4
             2          1              5
                        2              4
                                      ..
11           2          2             73
                        3             58
                        4             15
                        5              3
                        6              3
Length: 101, dtype: int64
NEWHHFAMINC  NEWURBRUR  NEWNUMADLT  HOUSEID   
1            1          1           9000098169    1
                                    9000172032    2
                                    9000215818    2
                                    9000216337    1
                        2           9000017013    2
                                                 ..
11           2          4           9000072200    6
                                    9000167305    1
                         

In [31]:
#### Finally, we calculate total future trips here
total_new_trips = 0
count_miss_hh = 0
count_included_hh = 0
size = 0
lazy_count = 0
for key_inc in array_hhfaninc: 
    for key_urb in array_urbur:
        for key_adult in array_numadlt:
            try:
                average = ccm_mean_current[key_inc, key_urb, key_adult] 
                total_new_trips += ccm_hh_new[key_inc, key_urb, key_adult].size * average
                count_included_hh += 1
                size += ccm_hh_new[key_inc, key_urb, key_adult].size

            except:
                try:
                    lazy_count += ccm_hh_new[key_inc, key_urb, key_adult]
                    count_miss_hh += 1# This count allows us to know, how many groups in the future will not have inf in current data
                except:
                    print("Double Ghost")

print ("Number of HH groups with valid data from current time period: ", count_included_hh)
print ("Number of HH groups without data from current time period: ", count_miss_hh)
print (f"Number of future trips based on only:  {total_new_trips:.2f}")

print (f"Average number of trips per group:  {ccm_mean_new.mean():.2f}")

print (f"Total number of trips assuming average number of trips for missed groups: {(total_new_trips + count_miss_hh * ccm_mean_new.mean()):.2f}")
print ("Number of hoseholds accounted for: ", size)

Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Double Ghost
Number of HH groups with valid data from current time period:  87
Number of HH groups without data from current time period:  14
Number of future trips based on only:  4224.41
Average number of trips per group:  2.36
Total number of trips assuming average number of trips for missed groups: 4257.44
Number of ho

# Assignment
Use approach designed in this assignment to predict future number of trips for the shopping trips.
Use the following factors to create categories
- number of cars (column HHVEHCNT)
- hh size (column HHSIZE)

For the future, assume that half of the HH have 1 more person and 20% of hh have one more car


In [32]:
data_assigenment = da[da['TRIPPURP'] == 2]
display(data_assigenment.describe())

keys_HHsize = data_assigenment.HHSIZE.unique()
keys_HHV = data_assigenment.HHVEHCNT.unique()
display(keys_HHsize,keys_HHV,max(keys_HHsize),max(keys_HHV))

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,NEWHHFAMINC,NEWNUMADLT,NEWURBRUR
count,6.884000e+03,6884.000000,6884.000000,6884.000000,6.884000e+03,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.0,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.0,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6.884000e+03,6.884000e+03,6.884000e+03,6.884000e+03,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000
mean,9.000116e+09,1.487507,2.606769,2.605026,8.180378e+11,1.024840,1.350378,0.874637,1.628268,3.865921,1.999274,35.117664,1.994189,2.0,7.296485,26.131610,17.793579,1429.345874,1454.174172,0.986345,1.188553,3.394538,1.710488,1.133062,1.010459,-0.069436,-0.433033,-0.761621,-0.915601,-0.974288,-0.987507,-0.997385,-1.0,0.097037,1.613451,2.763074,1.102411,1.546049,3.145410,-0.981900,-0.993027,45.491284,6.334544,0.417054,6.899803,7.677041e+06,4.736789e+06,2.997874e+06,9.000116e+13,4.318417,398.829024,5.718332,2.008135,1.699448,1.767868,5.186084,2.670105,1.922574,1.904707,54.108077,7.172138,2.730244,2.539367,2.044306,6.047647,2.247240,3.564207,1.590936,4.113887,1.168361,202211.553021,1.104445,1029.563916,51.522516,1.442766,1.336142,0.943928,2.971528,1.917926,1.200174,5.052150,6.177949,1.513800,1.942621,7.294160,3.084834,3.061592,1.522952,32.313335,4.268884,3.304474,5.083236,32.919378,4.335270,3.372603,31.754503,4.249274,3.288059,5.034282,32.035445,4.319436,3.349942,7.172138,2.008135,1.168361
std,5.823645e+04,0.797709,1.781649,1.779795,2.589737e+11,0.155649,1.224175,0.587141,0.483302,2.119256,0.026943,78.135215,0.076011,0.0,5.785375,30.372081,26.983757,366.166097,369.495924,0.442989,0.943813,4.483240,0.930382,0.339667,1.008553,1.289257,1.103014,0.771215,0.474233,0.265299,0.185531,0.088536,0.0,0.403238,0.850464,12.766028,0.716830,0.818428,13.791696,0.387416,0.117893,680.254367,5.776235,3.331456,14.767990,9.649314e+06,8.473801e+06,6.342475e+06,5.823645e+08,12.011970,69.782324,3.407371,0.723563,0.813688,0.422224,2.471885,1.002235,0.267285,0.768920,24.748538,2.822952,12.137297,1.304021,1.077114,3.466457,0.843844,1.530295,1.118445,1.651925,0.374214,19.866025,0.942322,4.947683,19.368422,0.919580,0.752174,0.501973,13.007181,0.274498,0.400160,2.067166,21.161902,0.499846,1.440492,2.480648,1.252193,1.282652,1.059564,20.828039,1.94771

array([4, 2, 3, 1, 5, 8, 6, 7, 9])

array([ 2,  1,  3,  0,  5,  4,  6,  7,  8, 11])

np.int64(9)

np.int64(11)

In [ ]:
group_current = data_assigenment.groupby(['HHVEHCNT','HHSIZE']).size()
group_current_mean = data_assigenment.groupby(['HHVEHCNT','HHSIZE','HOUSEID']).size().groupby(level=[0,1]).mean()

np.int64(6884)

In [34]:
#create the new columns
data_assigenment = data_assigenment.copy()
data_assigenment['NewHHSIZE'] = data_assigenment.loc[:,'HHSIZE']
data_assigenment['NewHHVEHCNT'] = data_assigenment.loc[:,'HHVEHCNT']

# demonstrate
display(data_assigenment.head(10))

#identify the HOUSEID
HH_IDs = data_assigenment['HOUSEID'].unique()
HH_IDs

,HOUSEID,PERSONID,TRIPID,SEQ_TRIPID,VEHCASEID,FRSTHM,PARK,HHMEMDRV,TDWKND,TRAVDAY,LOOP_TRIP,DWELTIME,PUBTRANS,TRIPPURP,WHYFROM,WHYTRP1S,TRVLCMIN,STRTTIME,ENDTIME,TRPHHVEH,VEHID,TRPTRANS,NUMONTRP,ONTD_P1,ONTD_P2,ONTD_P3,ONTD_P4,ONTD_P5,ONTD_P6,ONTD_P7,ONTD_P8,ONTD_P9,ONTD_P10,NONHHCNT,HHACCCNT,WHODROVE,DRVR_FLG,PSGR_FLG,WHODROVE_IMP,PARK2_PAMOUNT,PARK2_PAYTYPE,PARK2,WHYTO,WALK,TRPMILES,WTTRDFIN,WTTRDFIN5D,WTTRDFIN2D,TDCASEID,VMT_MILE,GASPRICE,WHYTRP90,NUMADLT,HOMEOWN,RAIL,CENSUS_D,CENSUS_R,HH_HISP,DRVRCNT,CDIVMSAR,HHFAMINC,HH_RACE,HHSIZE,HHVEHCNT,LIF_CYC,MSACAT,MSASIZE,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,R_AGE,R_SEX,WORKER,DRIVER,R_RACE,R_HISP,PROXY,EDUC,PRMACT,R_SEX_IMP,VEHTYPE,HHFAMINC_IMP,OBHUR,DBHUR,TRIPMODE,OTHTNRNT,OTPPOPDN,OTRESDN,OTEEMPDN,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN,NEWHHFAMINC,NEWNUMADLT,NEWURBRUR,NewHHSIZE,NewHHVEHCNT
2,9000013002,2,1,1,900001300202,1,-1,1,1,1,2,10,2,2,1,80,30,700,730,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,12,-1,17.077067,2.208973e+06,0.000000e+00,2.274239e+06,90000130020201,17.077067,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,4,2,1,20,4,2,5,5,4,3,40,5,4,6,60,4,3,11,2,1,4,2
4,9000013002,2,3,3,900001300202,1,-1,1,1,1,2,-9,2,2,12,1,30,800,830,1,2,3,1,2,1,2,2,-1,-1,-1,-1,-1,-1,0,1,2,1,2,2,-1.0,-1,-1,1,-9,14.098819,2.208973e+06,0.000000e+00,2.274239e+06,90000130020203,14.098819,355.1,10,2,1,1,1,1,2,2,11,11,1,4,2,4,1,5,1,4,1,202202,1,1021,42,1,1,1,1,2,2,7,-1,1,3,11,2,4,1,5,3,2,4,5,3,2,20,4,2,5,5,4,3,11,2,1,4,2
13,9000013039,1,1,1,900001303901,1,2,1,2,5,2,50,2,2,1,40,10,1700,1710,1,1,1,1,1,2,2,2,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,13,1,0.635177,4.905966e+06,3.601782e+06,0.000000e+00,90000130390101,0.635177,341.9,3,2,3,1,1,1,2,2,11,7,1,4,1,6,1,4,1,4,1,202201,1,1021,38,1,1,1,1,2,1,6,-1,1,1,7,5,5,1,60,7,6,8,60,7,2,60,3,6,8,80,7,6,7,2,1,4,1
14,9000013039,1,2,2,900001303901,1,2,1,2,5,2,-9,2,2,13,1,10,1800,1810,1,1,1,1,1,2,2,2,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,1,1,0.635177,4.905966e+06,3.601782e+06,0.000000e+00,90000130390102,0.635177,341.9,3,2,3,1,1,1,2,2,11,7,1,4,1,6,1,4,1,4,1,202201,1,1021,38,1,1,1,1,2,1,6,-1,1,1,7,5,5,1,60,3,6,8,80,7,6,60,7,6,8,60,7,2,7,2,1,4,1
28,9000013045,1,5,5,900001304501,1,2,1,2,2,2,-9,2,2,13,1,15,1705,1720,1,1,1,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,0,2,1,1,2,1,-1.0,-1,-1,1,1,3.706029,1.834836e+06,1.457483e+06,0.000000e+00,90000130450105,3.706029,355.1,3,2,3,1,1,1,2,1,11,6,1,2,1,2,1,5,1,4,1,202202,2,1021,35,1,1,1,1,2,1,8,-1,1,1,6,5,5,1,60,8,7,8,60,8,7,50,6,5,8,50,7,6,6,2,1,2,1
31,9000013045,2,3,3,900001304501,1,-1,1,2,2,2,-9,2,2,13,1,15,1705,1720,1,1,1,2,1,1,-1,-1,-1,-1,-1,-1,-1,-1,0,2,1,2,1,1,-1.0,-1,-1,1,1,3.706029,2.008969e+06,1.713049e+06,0.000000e+00,90000130450203,-1.000000,355.1,5,2,3,1,1,1,2,1,11,6,1,2,1,2,1,5,1,4,1,202202,2,1021,34,2,1,2,1,2,2,8,-1,2,1,6,5,5,2,60,8,7,8,60,8,7,50,6,5,8,50,7,6,6,2,1,2,1
40,9000013081,1,3,3,900001308101,1,2,1,2,5,2,-9,2,2,13,1,20,1315,1335,1,1,3,1,1,2,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,1,1,8.615289,3.289997e+06,5.962095e+06,0.000000e+00,90000130810103,8.615289,344.9,6,2,2,2,1,1,2,2,13,6,1,2,3,10,3,3,4,6,2,202202,1,1022,62,2,2,1,1,2,1,3,5,2,3,6,2,1,1,30,1,2,5,50,4,3,20,2,1,2,30,2,1,6,2,2,2,3
46,9000013099,1,1,1,900001309901,1,2,1,2,4,2,4,2,2,1,80,8,830,838,1,1,1,1,1,2,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,12,2,3.704164,2.609425e+06,2.474083e+06,0.000000e+00,90000130990101,3.704164,350.2,10,2,1,1,1,1,2,2,11,11,1,2,2,2,1,4,1,4,1,202202,2,1021,38,1,1,1,1,2,1,6,-1,1,1,11,4,2,1,20,5,3,6,1,4,2,20,1,2,5,5,3,2,11,2,1,2,2
47,9000013099,1,2,2,900001309901,1,2,1,2,4,2,-9,2,2,12,1,8,842,850,1,1,1,1,1,2,-1,-1,-1,-1,-1,-1,-1,-1,0,1,1,1,2,1,-1.0,-1,-1,1,1,3.013052,2.609425e+06,2.474083e+06,0.000000e+00,90000130990102,3.013052,350.2,10,2,1,1,1,1,2,2,11,11,1,2,2,2,1,4,1,4,1,202202,2,1021,38,1,1,1,1,2,1,6,-1,1,1,11,2,4,1,20,1,2,5,5,3,2,20,5,3,6,1,4,2,11,2,1,2,2
48,9000013122,1,1,1,-1,2,2,-1,1,7,2,55,2,2,1,

array([9000013002, 9000013039, 9000013045, ..., 9000217969, 9000218006,
       9000218013])

In [35]:
# For 20% of HH will have 1 more vehicle
HHV_increase = pd.Series(HH_IDs).sample(frac=0.2,random_state=1).tolist()
increase_HHV = data_assigenment['HOUSEID'].isin(HHV_increase)
data_assigenment.loc[increase_HHV,'NewHHVEHCNT'] = np.minimum(data_assigenment.loc[increase_HHV,'NewHHVEHCNT'] + 1, 11)

# For half of HH will have 1 more person in the future
HHsize_increse = pd.Series(HH_IDs).sample(frac=0.5,random_state=1).tolist()
increase_HHsize = data_assigenment['HOUSEID'].isin(HHsize_increse)
data_assigenment.loc[increase_HHsize,'NewHHSIZE'] = np.minimum(data_assigenment.loc[increase_HHsize,'NewHHSIZE'] + 1, 9)


In [36]:
newgroup = data_assigenment.groupby(['NewHHVEHCNT','NewHHSIZE']).size()
newgroup_HH = data_assigenment.groupby(['NewHHVEHCNT','NewHHSIZE','HOUSEID']).size()
newgroup_mean = data_assigenment.groupby(['NewHHVEHCNT','NewHHSIZE','HOUSEID']).size().groupby(level=[0,1]).mean()
display(newgroup,newgroup_HH,newgroup_mean)

NewHHVEHCNT  NewHHSIZE
0            1            55
             2            83
             3            38
             4             6
             5            10
                          ..
8            4             2
             8             2
9            2             1
11           3             1
             4             1
Length: 64, dtype: int64

NewHHVEHCNT  NewHHSIZE  HOUSEID   
0            1          9000013565    3
                        9000015004    2
                        9000018365    2
                        9000018944    2
                        9000048308    2
                                     ..
8            4          9000171278    2
             8          9000073327    2
9            2          9000057364    1
11           3          9000098867    1
             4          9000209529    1
Length: 2872, dtype: int64

NewHHVEHCNT  NewHHSIZE
0            1            1.718750
             2            2.243243
             3            2.533333
             4            1.500000
             5            2.500000
                            ...   
8            4            2.000000
             8            2.000000
9            2            1.000000
11           3            1.000000
             4            1.000000
Length: 64, dtype: float64

In [37]:
# Calculate the future shopping trips

ttrips = 0
inhere_HH = 0
disapper_group_HH = 0
disapper_HH = 0
HHsize = 0
for key_hhv in keys_HHV:
    for key_hhsize in keys_HHsize:
        try:
            avg = group_current_mean[key_hhv,key_hhsize]
            ttrips += newgroup_HH[key_hhv,key_hhsize].size * avg
            inhere_HH += 1
            HHsize += newgroup_HH[key_hhv,key_hhsize].size
        except:
            try:
                disppear_HH = newgroup_HH[key_hhv,key_hhsize]
                disapper_group_HH += 1         
            except:
                print('void')



print(f"The total number of shopping trips in the future: {ttrips:.3f}")
print(f"The total number of Household group still exist in the future: {inhere_HH}")
print(f"The total number of Household group will disappear in the future: {disapper_group_HH}")
print(f"The total number of Household accouted for: {HHsize}")
print(f"The average shopping trips for Household group in the future: {newgroup_HH.mean().round(3)}")
print(f"The total number of trips include disappeared Household : {(ttrips+disapper_group_HH*newgroup_HH.mean()):5.3f}")

void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
void
The total number of shopping trips in the future: 7271.860
The total number of Household group still exist in the future: 52
The total number of Household group will disappear in the future: 11
The total number of Household accouted for: 2847
The average shopping trips for Household group in the future: 2.397
The total number of trips include disappeared Household : 7298.226


# To submit your homework:

    1 - Run all code cells 

    2 - Answer all questions

    3 - Download as pdf (you can use print the page and save it as pdf)  

    4 - Inspect the pdf file (all cells are executed and all questions are answered?)

    5 - If it looks good

            - Rename it as following *WA7_first_last* replacing *first_last* with your first and last names
    
            - Upload pdf to UBLearns/BrightSpace
    
        else
    
            - Fix the issue
    
            - Repeat from step 3
    

<div class="alert alert-block alert-info">
<b>Tip:</b> you may need to install additional libriaries to enable download into pdf funciton using pip and command window, specifically: >pyppeteer-install and/or nbconvert[webpdf].
</div>